In [1]:
import pickle
from collections import defaultdict
import os
from tqdm.notebook import tqdm
from pyaspeller import YandexSpeller
import codecs
import json
import requests
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import functools
from string import punctuation
from re import escape
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [2]:
with open("containers2.pkl","rb") as f:
    containers2 = pickle.load(f)
with open("containers1.pkl","rb") as f:
    containers1 = pickle.load(f)
with open("containers3.pkl","rb") as f:
    containers3 = pickle.load(f)

In [3]:
word_lems = {}
for i in tqdm(containers2.keys()):
    for line in containers2[i]:
        for word in line.split():
            if word not in word_lems.keys():
                word_lems[word] = morph.parse(word)[0].normal_form 

In [4]:
for i in tqdm(containers1.keys()):
    for line in containers1[i]:
        for word in line.split():
            if word not in word_lems.keys():
                word_lems[word] = morph.parse(word)[0].normal_form 

In [5]:
for i in tqdm(containers3.keys()):
    for line in containers3[i]:
        for word in line.split():
            if word not in word_lems.keys():
                word_lems[word] = morph.parse(word)[0].normal_form 

In [6]:
all_lemms = set()
with codecs.open('Полная парадигма. Морфология. Орфоэпия. Частотность.txt',encoding = 'Windows-1251') as f:
    i = 0
    for line in tqdm(f):
        i+=1
        if line[0] != ' ':
            for k in line.split('|')[0].split():
                all_lemms.add(k)
lems = set()
for word in tqdm(all_lemms):
    lems.add(morph.parse(word)[0].normal_form)

In [7]:
len(word_lems.keys())

323844

In [8]:
counter = 0
er = set()
for word in word_lems.keys():
    if word_lems[word] not in lems and not word.isdigit():
        counter += 1
        er.add(word_lems[word])

In [9]:
len(er)

105625

In [10]:
def illustration(func):
    """
    Распаралеливание выкачки страниц.
    """
    mutex = ThreadLock()
    n_thread = ThreadValue('i',0)
    @functools.wraps(func)
    def wrapper(*args, **argv):
        result = func(*args, **argv)
        with mutex:
            nonlocal n_thread
            n_thread.value +=1
            if n_thread.value % 10 == 0:
                print(f"\r{n_thread.value} objects are processed...",end ='',flush = True)
        return result
    return wrapper
mistakes = []
@illustration
def change_mistakes(string):
    try:
        req = requests.post('https://speller.yandex.net/services/spellservice.json/checkText',data = {'text':string})
        new_string = string
        minus_len = 0
        for word in req.json():
            new_string = new_string[:word['pos'] - minus_len] + word['s'][0] + new_string[word['pos'] - minus_len + word['len']:]
            minus_len += word['len'] - len(word['s'][0]) 
        n_string = ' '.join([morph.parse(word)[0].normal_form for word in new_string.split()])
        error[string] = n_string
    except:
        print('ERROR!')
        mistakes.append(string)
error = {}
with ThreadPool(20) as pool: 
    pool.map(change_mistakes, er)

14200 objects are processed...ERROR!
30330 objects are processed...ERROR!
46480 objects are processed...ERROR!
62600 objects are processed...ERROR!
78710 objects are processed...ERROR!
94820 objects are processed...ERROR!
105620 objects are processed...

In [11]:
mistakes

['ррахиима', '7ч', 'шагинян', 'января4', 'приколыфнаф', 'бойли']

In [18]:
change_mistakes('бойли')

In [27]:
new_docs = {}
for i in tqdm(containers2.keys()):
    lines = []
    for line in containers2[i]:
        tmp = ''
        for word in line.split():
            if word_lems[word] in error.keys() :
                tmp += error[word_lems[word]] + ' '
            else:
                tmp += word_lems[word] + ' '
        lines.append(tmp)
    new_docs[i] = lines

In [28]:
for i in tqdm(containers1.keys()):
    lines = []
    for line in containers1[i]:
        tmp = ''
        for word in line.split():
            if word_lems[word] in error.keys() :
                tmp += error[word_lems[word]] + ' '
            else:
                tmp += word_lems[word] + ' '
        lines.append(tmp)
    new_docs[i] = lines

In [29]:
for i in tqdm(containers3.keys()):
    lines = []
    for line in containers3[i]:
        tmp = ''
        for word in line.split():
            if word_lems[word] in error.keys() :
                tmp += error[word_lems[word]] + ' '
            else:
                tmp += word_lems[word] + ' '
        lines.append(tmp)
    new_docs[i] = lines

In [33]:
with open('containers.pkl','wb') as f:
    pickle.dump(new_docs,f)

In [32]:
len(new_docs.items())

38114